In [19]:
import numpy as np
import pandas as pd 
from zytholic_project.base_model import BaseModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import sigmoid_kernel

In [45]:
model.get_data()
model.set_preprocess_pipeline()
model.process_data()

/root/.pyenv/versions/lewagon/envs/zytholic_project/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [49]:
model.





,name,style,brewery,abv,ave rating,min ibu,max ibu,astringency,body,alcohol,...,sweet,sour,salty,fruits,hoppy,spices,malty,state,country,retired
0,Double Bag,Altbier,Long Trail Brewing Co.,7.2,3.90,25,50,12,57,18,...,55,16,0,24,35,12,84,VT,US,f
1,Long Trail Ale,Altbier,Long Trail Brewing Co.,5.0,3.58,25,50,14,37,6,...,43,11,0,10,54,4,62,VT,US,f
2,Scurry,Altbier,Off Color Brewing,5.3,3.67,25,50,21,69,10,...,120,14,0,19,36,15,218,IL,US,f
3,Sleigh'r Dark Doüble Alt Ale,Altbier,Ninkasi Brewing Company,7.2,3.78,25,50,25,51,26,...,45,9,1,11,51,20,95,OR,US,f
4,Okto Festival Ale,Altbier,Widmer Brothers Brewing Company,5.3,3.46,25,50,28,40,3,...,58,29,0,36,54,8,97,OR,US,f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4389,The First Snow Ale,Winter Warmer,RJ Rockers Brewing Company,6.0,3.63,35,50,15,31,23,...,54,43,0,54,14,140,58,SC,US,f
4390,Red Nose Winter Ale,Winter Warmer,Natty Greene's Pub & Brewing Co.,6.8,3.59,35,50,8,44,24,...,52,21,0,26,21,96,77,NC,US,f
4391,Fish Tale Winterfish,Winter Warmer,Fish Brewing Company / Fishbowl Brewpub,7.5,3.76,35,50,11,36,50,...,72,59,0,81,110,18,73,WA,US,f
4392,"He'Brew Hanukkah, Chanukah: Pass The Beer",Winter Warmer,Shmaltz Brewing Company,8.0,3.61,35,50,6,64,30,...,78,15,1,28,57,23,129,NY,US,f
